<h3> 1. Import needed libraries <h3>

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from pyGRNN import GRNN
from sklearn.cluster import KMeans
from lazypredict.Supervised import LazyRegressor

In [60]:
from sklearn.metrics import mean_absolute_error
from numpy import sqrt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, max_error, median_absolute_error, mean_absolute_error

def print_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print(f'Mean Squared Error (MSE):              {mse:.10f}')
    
    rmse = sqrt(mse)
    print(f'Root Mean Squared Error (RMSE):        {rmse:.10f}')
    
    mae = mean_absolute_error(y_true, y_pred)
    print(f'Mean Absolute Error (MAE):             {mae:.10f}')
    
    r2 = r2_score(y_true, y_pred)
    print(f'R-squared (R²):                        {r2:.10f}')
    
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.10f}')    
    
    me = max_error(y_true, y_pred)
    print(f'Max Error (ME):                        {me:.10f}')    
    
    med_ae = median_absolute_error(y_true, y_pred)
    print(f'Median Absolute Error (MedAE):         {med_ae:.10f}') 

<h3> 2. Read data </h3>

In [61]:
df = pd.read_csv('data/train.csv')
df.head(10)

,max_discharge_sum_r1,max_discharge_sum_r2_1,max_discharge_sum_r2_2,max_discharge_sum_r3,flood_sum_r1,flood_sum_r2_1,flood_sum_r2_2,flood_sum_r3,HW_inund_uncertainty,bridge_scour_uncertainty,travel_demand_uncertainty,restoration_capacity_uncertainty,dir_costs,lost_trips_costs_s_r,traffic_costs_s_r
0,585.67,1626.36,1917.92,638.86,8411.14,31506.11,22090.55,15338.74,1.03,1.14,0.76,6,33407655.48,63906033.49,-13876077.42
1,577.56,1643.06,1887.66,821.35,8428.71,31464.77,21934.20,17058.63,1.03,0.91,1.14,5,26222060.06,55359791.46,9384497.19
2,772.58,1672.02,1803.84,494.53,9338.14,31334.39,21498.21,13903.41,1.08,0.95,1.06,3,25265321.95,59332912.51,5139630.61
3,522.08,1653.12,3326.49,460.21,8055.62,40539.30,32937.77,13073.49,1.05,1.26,0.98,3,54860001.97,94212106.15,2127730.92
4,614.14,1644.18,2485.55,595.06,8529.56,34186.48,28422.80,14950.68,0.79,0.70,1.23,4,39212152.98,76321454.19,21132083.35
5,813.29,1669.06,1877.53,680.46,9701.22,31193.17,22297.85,15652.70,0.78,1.38,0.88,6,32258229.36,76162964.60,-6506723.41
6,663.21,1666.04,2230.32,671.19,8806.26,32504.62,26343.62,15618.30,0.88,0.74,0.64,6,35019441.00,32830011.84,-17554044.81
7,501.75,1726.18,2023.48,993.98,8031.13,32844.96,22531.50,18555.23,0.64,0.35,1.19,5,28815518.56,71389907.84,17423834.01
8,581.13,1669.38,1976.99,1105.23,8297.44,32213.13,22180.60,19413.06,0.97,0.52,0.94,6,31828474.78,56818034.31,-2533088.91
9,622.46,1636.29,2176.80,457.47,8532.34,32557.58,24402.16,13253.84,0.86,1.26,1.16,4,45656687.89,61532735.19,14043780.74


In [62]:
test_df = pd.read_csv('data/test.csv')

In [63]:
y1_name, y2_name, y3_name = "dir_costs","traffic_costs_s_r","lost_trips_costs_s_r" 
y1, y2, y3 = df[y1_name], df[y2_name], df[y3_name]
test_y1, test_y2, test_y3 = test_df[y1_name], test_df[y2_name], test_df[y3_name]

<h3> 3. Scale features </h3>

In [64]:
X = df.drop(columns=[y1_name, y2_name, y3_name])
scaler = MaxAbsScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

In [65]:
test_X = test_df.drop(columns=[y1_name, y2_name, y3_name])
X_test_scaled = scaler.transform(test_X)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=test_X.columns)

In [69]:
reg2 = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
_, predictions2 = reg2.fit(X, test_X, y2, test_y2)
predictions2

100%|██████████| 42/42 [00:06<00:00,  6.07it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2813
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 12
[LightGBM] [Info] Start training from score 2376948.061796


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
XGBRegressor,0.84,0.84,6061095.29,0.71
LGBMRegressor,0.83,0.84,6094770.44,0.06
ExtraTreesRegressor,0.82,0.82,6382617.83,0.57
HistGradientBoostingRegressor,0.82,0.82,6405705.96,0.32
BaggingRegressor,0.82,0.82,6424554.95,0.12
RandomForestRegressor,0.81,0.82,6441033.11,1.12
GradientBoostingRegressor,0.81,0.82,6521976.44,0.62
Lars,0.77,0.77,7240474.22,0.02
LassoLars,0.77,0.77,7241961.89,0.02


<p> <b> Міні-висновок: </b> Проблема не в масштабуванні даних. Масшатбування даних трохи покращує RMSE 5461130.73 -> 4821200.67 </p>

<h3> 4. Add clusters </h3>

<h4> a. train </h4>

In [40]:
k = 10
print("number of clusters = ", k)

kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_scaled_df)
labels = kmeans.labels_
# Add the cluster labels to the train DataFrame
X_scaled_clustered_df = X_scaled_df.copy(deep=True)
X_scaled_clustered_df['Cluster'] = labels
dummy_columns = pd.get_dummies(X_scaled_clustered_df['Cluster'], prefix='class')
X_scaled_clustered_df = pd.concat([X_scaled_clustered_df, dummy_columns], axis=1)
X_scaled_clustered_df = X_scaled_clustered_df.drop("Cluster", axis=1)

# Add the cluster labels to the test DataFrame
test_clusters = kmeans.predict(X_test_scaled_df)
X_test_scaled_clustered_df = X_test_scaled_df.copy(deep=True)
X_test_scaled_clustered_df["Cluster"] = test_clusters
test_dummy_columns = pd.get_dummies(X_test_scaled_clustered_df['Cluster'], prefix='class')
X_test_scaled_clustered_df = pd.concat([X_test_scaled_clustered_df, test_dummy_columns], axis=1)
X_test_scaled_clustered_df = X_test_scaled_clustered_df.drop("Cluster", axis=1)

number of clusters =  10


In [41]:
X_scaled_clustered_float64_df = X_scaled_clustered_df.astype(dtype=float)
X_test_scaled_clustered_float64_df = X_test_scaled_clustered_df.astype(dtype=float)

In [42]:
reg1 = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
_, predictions1 = reg1.fit(X_scaled_clustered_df, X_test_scaled_clustered_df, y1, test_y1)
predictions1

 74%|███████▍  | 31/42 [00:06<00:03,  3.63it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████| 42/42 [00:08<00:00,  4.89it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000170 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2813
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 12
[LightGBM] [Info] Start training from score 36977500.275990


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.87,0.88,4821200.67,0.54
GradientBoostingRegressor,0.87,0.88,4861268.94,0.60
RandomForestRegressor,0.86,0.87,4979378.80,1.20
HistGradientBoostingRegressor,0.86,0.87,5009826.73,0.22
LGBMRegressor,0.86,0.87,5050890.07,0.06
BaggingRegressor,0.86,0.87,5082296.41,0.13
SGDRegressor,0.85,0.85,5295384.27,0.02
OrthogonalMatchingPursuit,0.85,0.85,5302477.10,0.01
LassoCV,0.84,0.85,5316574.29,0.19


<p> <b> Міні-висновок: </b> Кластеризація ніяк не впливає на найкращий регресор</p>

<h4> 5. ADD GRNN1, GRNN2, GRNN3 to count variables multi-dependence</h4>

In [43]:
# 4. train GRNN for each dataset
grnn1 = GRNN()
grnn1.fit(X_scaled_clustered_df, y1)
y_pred1 = grnn1.predict(X_scaled_clustered_df)
print("Sigma1 = ", grnn1.sigma)

grnn2 = GRNN()
grnn2.fit(X_scaled_clustered_df, y2)
y_pred2 = grnn2.predict(X_scaled_clustered_df)
print("Sigma2 = ", grnn2.sigma)

grnn3 = GRNN()
grnn3.fit(X_scaled_clustered_df, y3)
y_pred3 = grnn3.predict(X_scaled_clustered_df)
print("Sigma3 = ", grnn3.sigma)

X1, X2, X3 = X_scaled_clustered_df.copy(deep=True), X_scaled_clustered_df.copy(deep=True), X_scaled_clustered_df.copy(deep=True)

X3["GRNN1"] = X2["GRNN1"] = X1["GRNN1"] = y_pred1
X3["GRNN2"] = X2["GRNN2"] = X1["GRNN2"] = y_pred2
X3["GRNN3"] = X2["GRNN3"] = X1["GRNN3"] = y_pred3

Executing warm start...
Warm start concluded. The optimum isotropic sigma is [0.07759374]
Gradient search concluded. The optimum sigma is [0.06431615 0.02068963 0.01609462 0.71081714 5.41707759 1.17477872
 0.03846657 1.03158035 0.33788862 0.97183265 0.43128598 0.82558112
 0.078      0.078      0.078      0.078      0.078      0.078
 0.078      0.078      0.078      0.078     ]
Sigma1 =  [0.06431615 0.02068963 0.01609462 0.71081714 5.41707759 1.17477872
 0.03846657 1.03158035 0.33788862 0.97183265 0.43128598 0.82558112
 0.078      0.078      0.078      0.078      0.078      0.078
 0.078      0.078      0.078      0.078     ]
Executing warm start...
Warm start concluded. The optimum isotropic sigma is [0.06845064]
Gradient search concluded. The optimum sigma is [1.56418311e-01 5.03200655e-02 6.08234926e-02 2.86035356e+01
 8.94409133e+00 6.19084082e+01 5.89682179e+01 3.74483815e+01
 1.07253541e+01 4.18497662e-01 2.27703347e-02 9.64677829e-02
 6.80000000e-02 6.80000000e-02 6.80000000e-02 6

<h3> 6. Scale one more time to count newly added GRNN features </h3>

In [51]:
# X1
X1_scaler = MaxAbsScaler()
X1_scaler.fit(X1)
X1_scaled = X1_scaler.transform(X1)

# X2
X2_scaler = MaxAbsScaler()
X2_scaler.fit(X2)
X2_scaled = X2_scaler.transform(X2)
 
# X3
X3_scaler = MaxAbsScaler()
X3_scaler.fit(X1)
X3_scaled = X3_scaler.transform(X3)

In [52]:
test_y_pred1 = grnn1.predict(X_test_scaled_clustered_df)
test_y_pred2 = grnn2.predict(X_test_scaled_clustered_df)
test_y_pred3 = grnn3.predict(X_test_scaled_clustered_df)
test_X1, test_X2, test_X3 = X_test_scaled_clustered_df.copy(deep=True), X_test_scaled_clustered_df.copy(deep=True), X_test_scaled_clustered_df.copy(deep=True)

In [53]:
test_X3["GRNN1"] = test_X2["GRNN1"] = test_X1["GRNN1"] = test_y_pred1
test_X3["GRNN2"] = test_X2["GRNN2"] = test_X1["GRNN2"] = test_y_pred2
test_X3["GRNN3"] = test_X2["GRNN3"] = test_X1["GRNN3"] = test_y_pred3

In [54]:
# test datasets (scaled)
test_X1_scaled = X1_scaler.transform(test_X1)
test_X2_scaled = X2_scaler.transform(test_X2) 
test_X3_scaled = X3_scaler.transform(test_X3)

In [56]:
reg2 = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
_, predictions2 = reg2.fit(X2_scaled, test_X2_scaled, y2, test_y2)
predictions2

100%|██████████| 42/42 [00:07<00:00,  5.30it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3608
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 25
[LightGBM] [Info] Start training from score 2376948.061796


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
DecisionTreeRegressor,0.81,0.82,6388993.49,0.04
BaggingRegressor,0.81,0.82,6446242.06,0.15
RandomForestRegressor,0.81,0.82,6459960.26,1.41
ExtraTreesRegressor,0.80,0.82,6522479.90,0.81
OrthogonalMatchingPursuit,0.80,0.81,6547122.79,0.01
OrthogonalMatchingPursuitCV,0.80,0.81,6547122.79,0.01
LGBMRegressor,0.80,0.81,6563509.22,0.08
SGDRegressor,0.80,0.81,6571018.69,0.04
LassoCV,0.80,0.81,6572890.31,0.17
